In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNetCV
from sklearn.impute import SimpleImputer
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

def process_full_oil_data(input_file='/Users/arthurf/Desktop/INDENG 242A/shortened_oil_data.csv', output_file='/Users/arthurf/Desktop/INDENG 242A/final_daily_processed_data.csv'):
    print("Step 1: Loading Data...")
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"Error: {input_file} not found. Please upload the file.")
        return

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    
    # ---------------------------------------------------------
    # Step 2: Weekly Feature Processing (Logic from Analysis_Weekly_Features_v2.ipynb)
    # ---------------------------------------------------------
    print("Step 2: Processing Weekly Features...")
    
    # Identify Daily vs Weekly columns based on naming conventions
    daily_cols_potential = [
        'Brent_Oil', 'USD_CAD', 'Crude_Oil', 'DXY', 'Emerging_Mkt', 'Gold', 
        'Copper', 'Heating_Oil', 'Junk_Bond', 'Transportation', 'Natural_Gas',
        'Oil_Services', 'Gasoline', 'Inflation_Tips', 'Energy_Stocks', 'SP500', 
        'Oil_VIX', 'US10Y', 'Crack_Spread_321', 'Gold_Oil_Ratio', 'Copper_Oil_Ratio',
        'Transport_Oil_Ratio', 'Service_Oil_Ratio',
        'SMA_5', 'SMA_20', 'SMA_50', 'Dist_SMA_20', 'Dist_SMA_50',
        'BB_Upper', 'BB_Lower', 'BB_Width', 'BB_Position', 
        'RSI_14', 'RSI_28', 'MACD', 'MACD_Signal', 'MACD_Hist',
        'Ret_1d', 'Ret_5d', 'Ret_20d', 'Ret_60d',
        'Realized_Vol_20d', 'Realized_Vol_60d', 'Annual_Vol_20d',
        'Vol_Adj_Mom_20d', 'Vol_Adj_Mom_60d',
        'Price_Range_20d', 'ROC_5', 'ROC_20',
        'Oil_Lag1', 'Oil_Lag2', 'Oil_Lag3', 'Oil_Lag5',
        'RSI_14_Lag1', 'MACD_Hist_Lag1', 'Ret_1d_Lag1', 'Ret_1d_Lag2',
        'US_Stocks_Crude', 'US_Stocks_Ex_SPR_Crude', 'US_Stocks_Crude_SPR'
    ]
    
    weekly_cols = [col for col in df.columns if '4W_' in col or col.startswith('US_Stocks_') 
                   or col.startswith('US_') or col.startswith('PADD') or col.startswith('AK_')
                   or col.startswith('L48_') or col.startswith('New_') or col.startswith('Central_')
                   or col.startswith('Lower_') or col.startswith('Cushing_')]
    
    # Refine lists based on overlap
    weekly_cols = [col for col in weekly_cols if col not in daily_cols_potential]
    daily_cols = [col for col in daily_cols_potential if col in df.columns]
    
    # Create Week Identifier
    df['Year_Week'] = df['Date'].dt.isocalendar().year.astype(str) + '_W' + df['Date'].dt.isocalendar().week.astype(str).str.zfill(2)
    
    # Aggregate to Weekly: Daily cols take last value, Weekly cols take filled last value
    # We perform the aggregation logic
    def agg_logic(x):
        data = {}
        # For daily columns, take the last available value in the week
        for col in daily_cols:
            if col in x.columns:
                data[col] = x[col].iloc[-1]
        
        # For weekly columns, use backfill then take last to ensure data availability
        for col in weekly_cols:
            if col in x.columns:
                val = x[col].bfill().iloc[-1] if x[col].notna().any() else np.nan
                data[col] = val
        return pd.Series(data)

    df_weekly_agg = df.groupby('Year_Week').apply(agg_logic).reset_index()
    
    # Merge aggregated weekly values back to daily dataframe
    # This ensures "in the same week every day will be the same" for weekly vars
    
    # First, drop original weekly columns from df to avoid duplicates/confusion
    df_clean = df[['Date', 'Year_Week'] + daily_cols].copy()
    
    # Merge the aggregated weekly features
    # Suffix '_WeeklyAgg' is not needed if we replace, but to be safe we rename columns from aggregation
    # to differentiate or just overwrite. The prompt implies "attached", so let's overwrite the weekly columns with their weekly-constant versions.
    df_final = pd.merge(df_clean, df_weekly_agg.drop(columns=daily_cols, errors='ignore'), on='Year_Week', how='left')
    
    # Handle NaNs (Imputation)
    # Filter numeric columns for imputation
    numeric_cols = df_final.select_dtypes(include=[np.number]).columns
    imputer = SimpleImputer(strategy='median')
    df_final[numeric_cols] = imputer.fit_transform(df_final[numeric_cols])
    
    print(f"   Data merged. Shape: {df_final.shape}")

    # ---------------------------------------------------------
    # Step 3: Define Feature Groups (from PCA.ipynb & ENet.ipynb)
    # ---------------------------------------------------------
    
    # Fundamental
    fund_cols = df_final.columns[1:24].tolist() # Approximate from notebook logic
    # Refining based on explicit list if available in snippets or intersection
    fundamental_cols = [c for c in [
        'Brent_Oil','USD_CAD','DXY','Emerging_Mkt','Gold','Copper','Heating_Oil',
        'Junk_Bond','Transportation','Natural_Gas','Oil_Services','Gasoline',
        'Inflation_Tips','Energy_Stocks','SP500','Oil_VIX','US10Y'
    ] if c in df_final.columns]

    # Technical (Momentum + Volatility)
    mom_features = [f for f in ['RSI_14', 'RSI_28', 'ROC_5', 'ROC_20', 'Ret_5d', 'Ret_20d', 'Ret_60d',
                                'Vol_Adj_Mom_20d', 'Vol_Adj_Mom_60d'] if f in df_final.columns]
    vol_features = [f for f in ['BB_Width', 'Realized_Vol_20d', 'Realized_Vol_60d', 'Annual_Vol_20d', 'Price_Range_20d'] if f in df_final.columns]
    tech_cols = mom_features + vol_features
    
    # EIA Weekly
    eia_cols = [col for col in weekly_cols if col in df_final.columns]
    
    feature_groups = {
        'Fundamental': fundamental_cols,
        'Technical': tech_cols,
        'EIA': eia_cols
    }
    
    # Target for Elastic Net (Using Crude Oil Price as per ENet notebook, or Returns)
    # Notebook ENet (1).ipynb uses: y = df['Crude_Oil']
    y = df_final['Crude_Oil']

    # ---------------------------------------------------------
    # Step 4 & 5: Apply PCA and Elastic Net to Groups
    # ---------------------------------------------------------
    print("Step 3: Calculating PCA Components and Elastic Net Scores...")

    for name, cols in feature_groups.items():
        if len(cols) == 0:
            continue
            
        X_group = df_final[cols]
        
        # --- PCA ---
        # Standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_group)
        
        # Fit PCA
        pca = PCA(n_components=0.90) # Keep components explaining 90% variance
        X_pca = pca.fit_transform(X_scaled)
        
        # Attach PCs to dataframe
        n_pcs = X_pca.shape[1]
        # Limit to top 5 to avoid bloating if 90% requires many, or keep all as requested "pca resulting pcs"
        # We will keep up to 5 for clarity, or all if fewer.
        n_pcs_to_keep = min(n_pcs, 5) 
        
        for i in range(n_pcs_to_keep):
            df_final[f'PCA_{name}_PC{i+1}'] = X_pca[:, i]
            
        print(f"   Attached {n_pcs_to_keep} PCA components for {name}")

        # --- Elastic Net ---
        # The user wants "enet resulting stuff". We will calculate the ENet Prediction/Score
        # This creates a single synthetic feature representing the group's predictive power.
        try:
            enet_cv = ElasticNetCV(cv=3, random_state=42, max_iter=2000)
            enet_cv.fit(X_scaled, y)
            
            # Predict (Score)
            enet_score = enet_cv.predict(X_scaled)
            df_final[f'ENet_{name}_Score'] = enet_score
            
            # Also identifying selected features could be useful, but adding them as columns (flags) might be messy.
            # The Score is the most useful dense "derived" attribute.
            selected_feats = np.array(cols)[enet_cv.coef_ != 0]
            print(f"   Attached ENet Score for {name} (Selected {len(selected_feats)}/{len(cols)} features)")
            
        except Exception as e:
            print(f"   Skipping ENet for {name} due to error: {e}")

    # ---------------------------------------------------------
    # Step 6: Save
    # ---------------------------------------------------------
    print(f"Step 4: Saving final dataset to {output_file}...")
    df_final.to_csv(output_file, index=False)
    print("Done! File is ready.")
    
    return df_final

# Run the function
if __name__ == "__main__":
    # Create a dummy csv if not exists for demonstration or expect user file
    # This block assumes the environment has the file.
    process_full_oil_data()

Step 1: Loading Data...
Step 2: Processing Weekly Features...
   Data merged. Shape: (1229, 468)
Step 3: Calculating PCA Components and Elastic Net Scores...
   Attached 4 PCA components for Fundamental
   Attached ENet Score for Fundamental (Selected 14/17 features)
   Attached 5 PCA components for Technical
   Attached ENet Score for Technical (Selected 10/14 features)
   Attached 5 PCA components for EIA
   Attached ENet Score for EIA (Selected 0/406 features)
Step 4: Saving final dataset to /Users/arthurf/Desktop/INDENG 242A/final_daily_processed_data.csv...
Done! File is ready.
